<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="../../resources/logo.png" alt="Intellinum Bootcamp" style="width: 600px; height: 163px">
</div>

# Streaming ETL

Apache Spark&trade; makes it easy to build scalable and fault-tolerant streaming ETL applications.

## In this lesson you:
* Define logic to read from a stream of data
* Perform a basic ETL job on a streaming data source
* Join a stream to historical data
* Write to an always up-to-date Delta table


Run the following cell to create the lab environment:

In [327]:
#MODE = "LOCAL"
MODE = "CLUSTER"

import sys
from pyspark.sql import SparkSession
from pyspark import SparkConf
import os
from pyspark.sql import SparkSession
from pyspark.sql import Row
from pyspark import SparkConf
from pyspark.sql.types import *
from pyspark.sql import functions as F
from pyspark.storagelevel import StorageLevel
from matplotlib import interactive
interactive(True)
import matplotlib.pyplot as plt
%matplotlib inline
import json
import math
import numbers
import numpy as np
import plotly
import uuid
import time
plotly.offline.init_notebook_mode(connected=True)

sys.path.insert(0,'../../src')
from settings import *

try:
    fh = open('../../libs/pyspark24_py36.zip', 'r')
except FileNotFoundError:
    !aws s3 cp s3://devops.intellinum.co/bins/pyspark24_py36.zip ../../libs/pyspark24_py36.zip

try:
    spark.stop()
    print("Stopped a SparkSession")
except Exception as e:
    print("No existing SparkSession detected")
    print("Creating a new SparkSession")

SPARK_DRIVER_MEMORY= "1G"
SPARK_DRIVER_CORE = "1"
SPARK_EXECUTOR_MEMORY= "1G"
SPARK_EXECUTOR_CORE = "1"
SPARK_EXECUTOR_INSTANCES = 12



conf = None
if MODE == "LOCAL":
    os.environ["PYSPARK_PYTHON"] = "/home/yuan/anaconda3/envs/pyspark24_py36/bin/python"
    conf = SparkConf().\
            setAppName("pyspark_etl_12-streaming-etl").\
            setMaster('local[*]').\
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars', '../../libs/mysql-connector-java-5.1.45-bin.jar').\
            set('spark.jars.packages','net.java.dev.jets3t:jets3t:0.9.0,com.google.guava:guava:16.0.1')
else:
    os.environ["PYSPARK_PYTHON"] = "./MN/pyspark24_py36/bin/python"
    conf = SparkConf().\
            setAppName("pyspark_etl_12-streaming-etl-rj").\
            setMaster('yarn-client').\
            set('spark.executor.cores', SPARK_EXECUTOR_CORE).\
            set('spark.executor.memory', SPARK_EXECUTOR_MEMORY).\
            set('spark.driver.cores', SPARK_DRIVER_CORE).\
            set('spark.driver.memory', SPARK_DRIVER_MEMORY).\
            set("spark.executor.instances", SPARK_EXECUTOR_INSTANCES).\
            set('spark.sql.files.ignoreCorruptFiles', 'true').\
            set('spark.yarn.dist.archives', '../../libs/pyspark24_py36.zip#MN').\
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars.packages','io.delta:delta-core_2.11:0.2.0,org.apache.spark:spark-streaming-kafka-0-10_2.11:2.4.2,org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.2,net.java.dev.jets3t:jets3t:0.9.0,com.google.guava:guava:16.0.1'). \
            set('spark.driver.maxResultSize', '0').\
            set('spark.jars', 's3://devops.intellinum.co/bins/mysql-connector-java-5.1.45-bin.jar')
        

spark = SparkSession.builder.\
    config(conf=conf).\
    getOrCreate()


sc = spark.sparkContext

sc.addPyFile('../../src/settings.py')

sc=spark.sparkContext
hadoop_conf = sc._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
hadoop_conf.set("mapreduce.fileoutputcommitter.algorithm.version", "2")

def display(df, limit=10):
    try:
        # For spark-core 
        result = df.limit(limit).toPandas()
    except Exception as e:
        # For structured-streaming
        stream_name = str(uuid.uuid1()).replace("-","")
        query = (
          df
            .writeStream
            .format("memory")        # memory = store in-memory table (for debugging only)
            .queryName(stream_name) # show = name of the in-memory table
            .trigger(processingTime='1 seconds') #Trigger = 1 second
            .outputMode("append")  # append
            .start()
        )
        while query.isActive:
            time.sleep(1)
            result = spark.sql(f"select * from {stream_name} limit {limit}").toPandas()
            print("Wait until the stream is ready...")
            if result.empty == False:
                break
        result = spark.sql(f"select * from {stream_name} limit {limit}").toPandas()
    
    return result

def untilStreamIsReady(name):
    queries = list(filter(lambda query: query.name == name, spark.streams.active))

    if len(queries) == 0:
        print("The stream is not active.")

    else:
        while (queries[0].isActive and len(queries[0].recentProgress) == 0):
            pass # wait until there is any type of progress

        if queries[0].isActive:
            queries[0].awaitTermination(5)
            print("The stream is active and ready.")
        else:
            print("The stream is not active.")
            
            
def dfTest(id, expected, result):
    assert str(expected) == str(result), "{} does not equal expected {}".format(result, expected)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40587)
Traceback (most recent call last):
  File "/home/hadoop/anaconda/envs/pyspark24_py36/lib/python3.6/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/hadoop/anaconda/envs/pyspark24_py36/lib/python3.6/site-packages/py4j/java_gateway.py", line 1067, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused
ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40587)
Traceback (most recent call last):
  File "/home/hadoop/anaconda/envs/pyspark24_py36/lib/python3.6/site-packages/py4j/java_gateway.py", line 929, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empt

No existing SparkSession detected
Creating a new SparkSession


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40587)

### ETL on Streaming Data

Spark Streaming enables scalable and fault-tolerant ETL operations that continuously clean and aggregate data before pushing it to data stores.  Streaming applications can also incorporate machine learning and other Spark features to trigger actions in real time, such as flagging potentially fraudulent user activity.  This lesson is meant as an introduction to streaming applications as they pertain to production ETL jobs.  

Streaming poses a number of specific obstacles. These obstacles include:<br><br>

* *End-to-end reliability and correctness:* Applications must be resilient to failures of any element of the pipeline caused by network issues, traffic spikes, and/or hardware malfunctions
* *Handle complex transformations:* applications receive many data formats that often involve complex business logic
* *Late and out-of-order data:* network issues can result in data that arrives late and out of its intended order
* *Integrate with other systems:* Applications must integrate with the rest of a data infrastructure

Streaming data sources in Spark offer the same DataFrames API for interacting with your data.  The crucial difference is that in structured streaming, the DataFrame is unbounded.  In other words, data arrives in an input stream and new records are appended to the input DataFrame.

<div><img src="../../resources/structured-streamining-model.png" style="height: 400px; margin: 20px"/></div>

### Connecting to the Stream

As data technology matures, the industry has been converging on a set of technologies.  Apache Kafka and the AWS managed alternative Kinesis has become the ingestion engine at the heart of many pipelines.  

This technology brokers messages between producers, such as an IoT device writing data, and consumers, such as a Spark cluster reading data to perform real time analytics. There can be a many-to-many relationship between producers and consumers and the broker itself is scalable and fault tolerant.

Connect to a Kafka topic that is streaming Wikipedia event data.

There are a number of ways to stream data.  One other common design pattern is to stream from an S3 bucket where any new files that appear will be read by the stream.  In this example, we'll be streaming directly from Kafka.

Start by defining the schema of the data in the stream.

In [218]:
from pyspark.sql.types import BooleanType, IntegerType, StructType, StringType, StructField, TimestampType

schema = (StructType()
  .add("timestamp", TimestampType())
  .add("url", StringType())
  .add("userURL", StringType())
  .add("pageURL", StringType())
  .add("isNewPage", BooleanType())
  .add("geocoding", StructType()
    .add("countryCode2", StringType())
    .add("city", StringType())
    .add("latitude", StringType())
    .add("country", StringType())
    .add("longitude", StringType())
    .add("stateProvince", StringType())
    .add("countryCode3", StringType())
    .add("user", StringType())
    .add("namespace", StringType()))
)

To read from a stream, use `spark.readStream()`, which returns a `DataStreamReader` class.  Then, configure the stream by adding the following options:<br><br>

* The server endpoint: `54.213.33.240:9092`
* The topic to subscribe to: `en`
* A location to log checkpoint metadata (more on this <a href='#Batch-and-Streaming-Writes'>below</a>)
* The format: `kafka` 

Finally, use the load method, which loads the data stream from the Kafka source and returns it as an unbounded `DataFrame`.

In [219]:
kafkaDF = (spark
  .readStream
  .option("kafka.bootstrap.servers", "54.213.33.240:9092")
  .option("subscribe", "en")
  .option("startingOffsets", "latest")
  .format("kafka")
  .load()
)

Kafka transmits information using a key, value, and metadata such as topic and partition.  The information we're interested in is the `value` column.  Since this is a binary value, we must first cast it to a `StringType`.  We must also provide it with a schema.  Finally, we can expand the full structure of the JSON.


Wait until the stream finishes initializing.  

Often streams are started with the `.start()` method.  In this example, `display()`, provided in Bootcamp environments, is running that command for you.

In [303]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StringType

kafkaCleanDF = (kafkaDF
  .select(from_json(col("value").cast(StringType()), schema).alias("message"))
  .select("message.*")
)

In [320]:
[q.stop() for q in spark.streams.active]

[None, None]

In [318]:
[x.name for x in spark.streams.active]

['49911c0a999d11e9b8910a370dd9e3a4', 'test_query']

In [324]:
display(kafkaCleanDF)

Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...


,timestamp,url,userURL,pageURL,isNewPage,geocoding
0,2019-06-28 13:23:49.688,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:Whispyhistory,http://en.wikipedia.org/wiki/Mike_Pringle_(doc...,False,"(None, None, None, None, None, None, None, Non..."
1,2019-06-28 13:23:49.748,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:2001:8004:15...,http://en.wikipedia.org/wiki/The_Entire_Histor...,False,"(AU, None, -25.0, Australia, -25.0, None, AUS,..."
2,2019-06-28 13:23:49.807,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:Toad02,http://en.wikipedia.org/wiki/Oceana_(non-profi...,False,"(None, None, None, None, None, None, None, Non..."
3,2019-06-28 13:23:49.866,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:Kvng,http://en.wikipedia.org/wiki/Bull_Pen_Pep_Band,False,"(None, None, None, None, None, None, None, Non..."
4,2019-06-28 13:23:49.928,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:S.A. Julio,http://en.wikipedia.org/wiki/2011���12_Gimn��s...,False,"(None, None, None, None, None, None, None, Non..."
5,2019-06-28 13:23:49.987,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:Twofingered ...,http://en.wikipedia.org/wiki/User_talk:Nehme1499,False,"(None, None, None, None, None, None, None, Non..."
6,2019-06-28 13:23:50.678,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:BrownHairedGirl,http://en.wikipedia.org/wiki/Edmonton-Highlands,False,"(None, None, None, None, None, None, None, Non..."
7,2019-06-28 13:23:50.737,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:Slambo,http://en.wikipedia.org/wiki/South_Island_line,False,"(None, None, None, None, None, None, None, Non..."
8,2019-06-28 13:23:50.796,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:37.228.231.144,http://en.wikipedia.org/wiki/Vehicle_registrat...,False,"(IE, Dublin, 53.333099365234375, Ireland, 53.3..."
9,2019-06-28 13:23:50.958,https://en.wikipedia.org/w/index.php?diff=9038...,http://en.wikipedia.org/wiki/User:146.115.131.36,http://en.wikipedia.org/wiki/Nassau_Broadcasti...,False,"(US, Newton, 42.35329818725586, United States,..."


Stop the stream

In [222]:
[q.stop() for q in spark.streams.active]

[None]

### Transform the Stream

We can now start to apply transformation logic to our data in real time.  Parse out only the non-null country data.

In [325]:
goeocodingDF = (kafkaCleanDF
  .filter(col("geocoding.country").isNotNull())
  .select("timestamp", "pageURL", "geocoding.countryCode2", "geocoding.city")
)

display(goeocodingDF)

Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...


,timestamp,pageURL,countryCode2,city
0,2019-06-28 13:29:52.516,http://en.wikipedia.org/wiki/Capital_punishmen...,SI,None


Stop the stream

In [307]:
dir(spark.streams.active[0])

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_jsq',
 'awaitTermination',
 'exception',
 'explain',
 'id',
 'isActive',
 'lastProgress',
 'name',
 'processAllAvailable',
 'recentProgress',
 'runId',
 'status',
 'stop']

In [309]:
spark.streams.active[0]

In [311]:
from pyspark.sql.streaming import StreamingQuery

In [314]:
?StreamingQuery

In [313]:
StreamingQuery?

In [224]:
[q.stop() for q in spark.streams.active]

[None]

### Combine with Historical Data

Joins between historical data and streams operate in much the same way that other joins work.  Join the stream of Wikipedia data on country metadata

Import a lookup table of country data.

In [225]:
countries = spark.read.parquet("s3a://data.intellinum.co/bootcamp/common/countries/ISOCountryCodes/ISOCountryLookup.parquet")

display(countries)

,EnglishShortName,alpha2Code,alpha3Code,numericCode,ISO31662SubdivisionCode,independentTerritory
0,Afghanistan,AF,AFG,004,ISO 3166-2:AF,Yes
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX,No
2,Albania,AL,ALB,008,ISO 3166-2:AL,Yes
3,Algeria,DZ,DZA,012,ISO 3166-2:DZ,Yes
4,American Samoa,AS,ASM,016,ISO 3166-2:AS,No
5,Andorra,AD,AND,020,ISO 3166-2:AD,Yes
6,Angola,AO,AGO,024,ISO 3166-2:AO,Yes
7,Anguilla,AI,AIA,660,ISO 3166-2:AI,No
8,Antarctica,AQ,ATA,010,ISO 3166-2:AQ,No
9,Antigua and Barbuda,AG,ATG,028,ISO 3166-2:AG,Yes


Join the two `DataFrame`s on their two-letter country code.

In [226]:
joinedDF = goeocodingDF.join(countries, goeocodingDF.countryCode2 == countries.alpha2Code)

display(joinedDF)

Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...


,timestamp,pageURL,countryCode2,city,EnglishShortName,alpha2Code,alpha3Code,numericCode,ISO31662SubdivisionCode,independentTerritory
0,2019-06-28 05:10:18.187,http://en.wikipedia.org/wiki/Special:Log/abuse...,BO,Santa Cruz,Bolivia (Plurinational State of),BO,BOL,068,ISO 3166-2:BO,Yes
1,2019-06-28 05:10:18.246,http://en.wikipedia.org/wiki/Jos��_Mar��a_Carr...,BO,Santa Cruz,Bolivia (Plurinational State of),BO,BOL,068,ISO 3166-2:BO,Yes
2,2019-06-28 05:10:18.480,http://en.wikipedia.org/wiki/United_States_men...,US,None,United States of America,US,USA,840,ISO 3166-2:US,Yes


Stop the stream

In [227]:
[q.stop() for q in spark.streams.active]

[None]

https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html

### Batch and Streaming Writes

The logic that applies to batch processing can be ported over to streaming often with only minor modifications.  One best practice is to run batch operations as streaming jobs using a single trigger.  By using a checkpoint location, the metadata on which data has already been processed will be maintained so the cluster can be shut down without a loss of information.  This works best on streaming from a directory, where new files that appear are added to the stream.

Writes can be done against always up-to-date parquet files or a Delta table, which offers ACID compliant transactions on top of parquet.

For more information on Delta Lake, see the <a href="https://delta.io/" target="_blank">Delta Lake Official website</a><br>
Read more about triggers in the <a href="https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#triggers" target="_blank">Structured Streaming Programming Guide</a>

Write to a Delta table and partition by country.

In [228]:
YOUR_FIRST_NAME = 'rajeev'
userhome = f"s3a://temp.intellinum.co/{YOUR_FIRST_NAME}"

In [229]:
!aws s3 ls temp.intellinum.co/rajeev/etl1p/

In [230]:
basePath = f"{userhome}/etl1p"
queryName = f"query_delta_lake_{str(uuid.uuid1()).replace('-','')}"
checkpointPath = f"{basePath}/joined.checkpoint.{queryName}"
joinedPath = "{}/joined".format(basePath)

delta_output_stream = (joinedDF
  .writeStream                                   # Write the stream
  .format("delta")                               # Use the delta format
  .partitionBy("countryCode2")                   # Specify a feature to partition on
  .option("checkpointLocation", checkpointPath)  # Specify where to log metadata
  .option("path", joinedPath)                    # Specify the output path
  .outputMode("append")                          # Append new records to the output path
  .queryName(queryName)                         # The name of the stream
  .start()                                       # Start the operation
)


In [231]:
!aws s3 ls temp.intellinum.co/rajeev/etl1p/

                           PRE joined.checkpoint.query_delta_lake_087336de996311e9b8910a370dd9e3a4/


In [232]:
untilStreamIsReady(queryName)

The stream is active and ready.


Check to see that the data is there.

In [233]:
from pyspark.sql.functions import count

countsDF = (spark.read.format("delta").load(joinedPath)
  .select(count("*").alias("count"))
)

display(countsDF)

,count
0,8


Check to see the output

In [234]:
!aws s3 ls {joinedPath.replace("s3a","s3")}/

                           PRE _delta_log/
                           PRE countryCode2=AU/
                           PRE countryCode2=IN/
                           PRE countryCode2=MY/
                           PRE countryCode2=PH/
                           PRE countryCode2=US/


Now stop all streams

In [235]:
[q.stop() for q in spark.streams.active]

[None]

### Failure Recovery

To recover from cluster failure, Spark uses checkpointing for maintaining state.  In our initial command, we accomplished this using `.option("checkpointLocation", checkpointPath)`.  The checkpoint directory is per query and while that query is active, Spark continuously writes metadata of the processed data to this checkpoint directory.  Even if the entire cluster fails, the query can be restarted on a new cluster using the same directory and Spark can use this to start a new query where the failed one left off.  

**This is how Spark ensures end-to-end, exactly-once guarantees and how Spark maintains metadata on what data it has already seen between streaming jobs.**

In [236]:
!aws s3 ls {checkpointPath.replace("s3a","s3")}/

                           PRE commits/
                           PRE offsets/
                           PRE sources/
2019-06-28 05:10:33         45 metadata


## Exercise 1: Streaming ETL

This exercise entails reading from a Kafka stream of product orders and writing the results to a Delta table.

### Step 1: Create a Streaming DataFrame

Create the streaming DataFrame `productsDF` using the following specifics:<br><br>

* Server endpoint: `54.213.33.240:9092`
* Topic name: `product-orders`
* Format: `kafka`

In [279]:
# TODO
productsDF = (spark
  .readStream
  .option("kafka.bootstrap.servers", "54.213.33.240:9092")
  .option("subscribe", "product-orders")
  .option("startingOffsets", "latest")
  .format("kafka")
  .load()
)

In [280]:
productsDF.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [281]:
display(productsDF)

Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...


,key,value,topic,partition,offset,timestamp,timestampType
0,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544529,1969-12-31 23:59:59.999,0
1,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544530,1969-12-31 23:59:59.999,0
2,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544531,1969-12-31 23:59:59.999,0
3,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544532,1969-12-31 23:59:59.999,0
4,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544533,1969-12-31 23:59:59.999,0
5,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544534,1969-12-31 23:59:59.999,0
6,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544535,1969-12-31 23:59:59.999,0
7,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544536,1969-12-31 23:59:59.999,0
8,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544537,1969-12-31 23:59:59.999,0
9,None,"[123, 34, 111, 114, 100, 101, 114, 73, 68, 34,...",product-orders,0,18544538,1969-12-31 23:59:59.999,0


In [282]:
# TEST - Run this cell to test your solution
from pyspark.sql.types import *

dfTest("ET3-P-02-01-01", True, productsDF.schema['key'].dataType in [BinaryType(), StringType()])
dfTest("ET3-P-02-01-02", True, productsDF.schema['value'].dataType in [BinaryType(), StringType()])
dfTest("ET3-P-02-01-03", True, productsDF.schema['topic'].dataType == StringType())
dfTest("ET3-P-02-01-04", True, productsDF.schema['partition'].dataType == IntegerType())
dfTest("ET3-P-02-01-05", True, productsDF.schema['offset'].dataType == LongType())
dfTest("ET3-P-02-01-06", True, productsDF.schema['timestamp'].dataType in [TimestampType(), StringType()])
dfTest("ET3-P-02-01-07", True, productsDF.schema['timestampType'].dataType == IntegerType())

print("Tests passed!")

Tests passed!


### Step 2: Defining a Schema

Define a schema that consists of the following values.  Save it as `productSchema`<br>

| Field            | Type             |
|:-----------------|:-----------------|
| `orderID`        | `IntegerType()`  |
| `productID`      | `IntegerType()`  |
| `orderTimestamp` | `TimestampType()`|
| `orderQty`       | `IntegerType()`  |

In [283]:
# TODO

productSchema = (StructType()
  .add("orderID", IntegerType())
  .add("productID", IntegerType())
  .add("orderTimestamp", TimestampType())
  .add("orderQty", IntegerType())
)

In [284]:
# TEST - Run this cell to test your solution.
from pyspark.sql.types import IntegerType, StringType, TimestampType

dfTest("ET3-P-02-02-01", True, productSchema['orderID'].dataType == IntegerType())
dfTest("ET3-P-02-02-02", True, productSchema['productID'].dataType == IntegerType())
dfTest("ET3-P-02-02-03", True, productSchema['orderTimestamp'].dataType in [StringType(), TimestampType()])
dfTest("ET3-P-02-02-04", True, productSchema['orderQty'].dataType == IntegerType())

print("Tests passed!")

Tests passed!


### Step 3: Parsing the Kafka Data

Parse the `value` column of the Kafka data.  Remember to use the `from_json()` function and apply your schema.  Save the results to `ordersDF`

In [285]:
# TODO
ordersDF = (productsDF
            .select(F.from_json(F.col("value").cast(StringType()), productSchema).alias("message"))
            .select("message.*")
#             .withColumnRenamed('orderID', 'order.orderID')
#             .withColumnRenamed('productID', 'order.productID')
#             .withColumnRenamed('orderTimestamp', 'order.orderTimestamp')
#             .withColumnRenamed('orderQty', 'order.orderQty')
           )

In [286]:
ordersDF.printSchema()

root
 |-- orderID: integer (nullable = true)
 |-- productID: integer (nullable = true)
 |-- orderTimestamp: timestamp (nullable = true)
 |-- orderQty: integer (nullable = true)



In [287]:
display(ordersDF)

Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...
Wait until the stream is ready...


,orderID,productID,orderTimestamp,orderQty
0,1190807,872,2019-06-28 05:19:13,7
1,1190808,829,2019-06-28 05:19:13,6
2,1190809,911,2019-06-28 05:19:13,6
3,1190810,890,2019-06-28 05:19:13,2
4,1190811,837,2019-06-28 05:19:13,6
5,1190812,870,2019-06-28 05:19:13,3
6,1190813,854,2019-06-28 05:19:13,5
7,1190814,804,2019-06-28 05:19:13,8
8,1190815,875,2019-06-28 05:19:13,10
9,1190816,923,2019-06-28 05:19:13,10


In [288]:
[q.stop() for q in spark.streams.active]

[None, None]

In [289]:
# TEST - Run this cell to test your solution.
from pyspark.sql.types import IntegerType, TimestampType

_df = ordersDF.select(col("order.orderID").alias("orderID"),
                       col("order.productID").alias("productID"),
                       col("order.orderTimestamp").alias("orderTimestamp"),
                       col("order.orderQty").alias("orderQty"))

expected = {"orderID": IntegerType(), "orderQty": IntegerType(), "orderTimestamp": TimestampType(), "productID": IntegerType()}
sch = {c.name: c.dataType for c in _df.schema}

dfTest("ET3-P-02-03-01", True, sch == expected)

print("Tests passed!")

AnalysisException: "cannot resolve '`order.orderID`' given input columns: [orderID, productID, orderTimestamp, orderQty];;\n'Project ['order.orderID AS orderID#279368, 'order.productID AS productID#279369, 'order.orderTimestamp AS orderTimestamp#279370, 'order.orderQty AS orderQty#279371]\n+- Project [message#279196.orderID AS orderID#279198, message#279196.productID AS productID#279199, message#279196.orderTimestamp AS orderTimestamp#279200, message#279196.orderQty AS orderQty#279201]\n   +- Project [jsontostructs(StructField(orderID,IntegerType,true), StructField(productID,IntegerType,true), StructField(orderTimestamp,TimestampType,true), StructField(orderQty,IntegerType,true), cast(value#279055 as string), Some(Etc/Universal)) AS message#279196]\n      +- StreamingRelationV2 org.apache.spark.sql.kafka010.KafkaSourceProvider@1d232152, kafka, Map(startingOffsets -> latest, subscribe -> product-orders, kafka.bootstrap.servers -> 54.213.33.240:9092), [key#279054, value#279055, topic#279

### Step 4: Write to a Delta Table

Write the results to an always up-to-date Delta table to the provided path.  The table should have the following columns:<br><br>

1. `orderID`
1. `productID`
1. `orderTimestamp`
1. `orderQty`

In [290]:
!aws s3 ls temp.intellinum.co/rajeev/etl1p/

In [291]:
# TODO
basePath = f"{userhome}/etl1p"
queryName = f"query_delta_lake_{str(uuid.uuid1()).replace('-','')}"
checkpointPath = f"{basePath}/orders.checkpoint.{queryName}"
ordersPath = "{}/orders".format(basePath)

# FILL_IN
delta_output_stream = (ordersDF
  .writeStream
  .format("delta")
  .option("checkpointLocation", checkpointPath)
  .option("path", ordersPath)
  .outputMode("append")
  .queryName(queryName)
  .start()
)


In [292]:
!aws s3 ls temp.intellinum.co/rajeev/etl1p/

                           PRE orders.checkpoint.query_delta_lake_4d74a410996411e9b8910a370dd9e3a4/


In [293]:
untilStreamIsReady(queryName)

The stream is active and ready.


In [294]:
!aws s3 ls {ordersPath.replace("s3a","s3")}/

                           PRE _delta_log/
2019-06-28 05:19:43       1427 part-00000-16ec59b8-f78c-44a1-9934-8dc1b7b24b33-c000.snappy.parquet
2019-06-28 05:19:52       4173 part-00000-224620c0-3b01-4b25-b1b4-260ddb2b692a-c000.snappy.parquet


In [295]:
temp = spark.read.format("delta").load(ordersPath)

In [296]:
temp.printSchema()

root
 |-- orderID: integer (nullable = true)
 |-- productID: integer (nullable = true)
 |-- orderTimestamp: timestamp (nullable = true)
 |-- orderQty: integer (nullable = true)



In [297]:
display(temp)

,orderID,productID,orderTimestamp,orderQty
0,1183763,838,2019-06-28 05:19:40,9
1,1183764,904,2019-06-28 05:19:40,7
2,1183765,765,2019-06-28 05:19:40,10
3,1183766,882,2019-06-28 05:19:40,1
4,1183767,810,2019-06-28 05:19:40,2
5,1183768,850,2019-06-28 05:19:40,5
6,1183769,854,2019-06-28 05:19:40,6
7,1183770,901,2019-06-28 05:19:40,1
8,1183771,844,2019-06-28 05:19:40,3
9,1183772,952,2019-06-28 05:19:40,8


In [298]:
# TEST - Run this cell to test your solution.
from pyspark.sql.types import *

ordersPath = "{}/orders".format(basePath)
_df = spark.read.format("delta").load(ordersPath)

expected = {"orderID": IntegerType(), "orderQty": IntegerType(), "orderTimestamp": TimestampType(), "productID": IntegerType()}
sch = {c.name: c.dataType for c in _df.schema}

dfTest("ET3-P-02-04-01", True, sch == expected)

print("Tests passed!")

Tests passed!


### Step 5: Now Stop the Stream and Delete Files

Stop the stream.

In [299]:
[q.stop() for q in spark.streams.active]

[None]

Recursively delete the files you created (this is a permanent operation).

In [300]:
basePath

's3a://temp.intellinum.co/rajeev/etl1p'

In [301]:
!aws s3 rm --recursive {basePath.replace('s3a','s3')}

delete: s3://temp.intellinum.co/rajeev/etl1p/orders.checkpoint.query_delta_lake_4d74a410996411e9b8910a370dd9e3a4/commits/0
delete: s3://temp.intellinum.co/rajeev/etl1p/orders.checkpoint.query_delta_lake_4d74a410996411e9b8910a370dd9e3a4/metadata
delete: s3://temp.intellinum.co/rajeev/etl1p/orders/_delta_log/00000000000000000001.json
delete: s3://temp.intellinum.co/rajeev/etl1p/orders.checkpoint.query_delta_lake_4d74a410996411e9b8910a370dd9e3a4/offsets/3
delete: s3://temp.intellinum.co/rajeev/etl1p/orders/_delta_log/00000000000000000002.json
delete: s3://temp.intellinum.co/rajeev/etl1p/orders.checkpoint.query_delta_lake_4d74a410996411e9b8910a370dd9e3a4/commits/1
delete: s3://temp.intellinum.co/rajeev/etl1p/orders.checkpoint.query_delta_lake_4d74a410996411e9b8910a370dd9e3a4/offsets/1
delete: s3://temp.intellinum.co/rajeev/etl1p/orders/part-00000-16ec59b8-f78c-44a1-9934-8dc1b7b24b33-c000.snappy.parquet
delete: s3://temp.intellinum.co/rajeev/etl1p/orders.checkpoint.query_delta_lake_4d74a410

## Review
**Question:** What is the best practice for different versions of ETL jobs?  
**Answer:** Generally speaking, an ETL solution should have three versions:
0. *Batch* or the ability to run a periodic workload
0. *Streaming* or the ability to process incoming data in real time
0. *Incremental* or the ability to process a specific set of data, especially in the case of job failure.<br>
In practice, batch and streaming jobs are oftentimes combined where a batch job is a streaming workload using a single trigger.

**Question:** What are commonly approached as data streams?  
**Answer:** Apache Kafka and the AWS managed alternative Kinesis are common data streams.  Additionally, it's common to monitor a directory for incoming files.  When a new file appears, it is brought into the stream for processing.

**Question:** How does Spark ensure exactly-once data delivery and maintain metadata on a stream?  
**Answer:** Checkpoints give Spark this fault tolerance through the ability to maintain state off of the cluster.

**Question:** How does the Spark approach to streaming integrate with other Spark features?  
**Answer:** Spark Streaming uses the same DataFrame API, allowing easy integration with other Spark functionality.

&copy; 2019 [Intellinum Analytics, Inc](http://www.intellinum.co). All rights reserved.<br/>